## import modules & set variables

In [ ]:
# for code implementation
import torch
import numpy as np
import torch.nn as nn
from google.colab import drive
from torch.optim import SGD, Adam
from torch.autograd import Variable
# for plotting graphs & data loader
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize
from torchvision.transforms import ToTensor, ToPILImage
# others
from argparse import ArgumentParser
# variables - fixed for VOC2012 data set
NUM_CHANNELS = 3
NUM_CLASSES = 22

## mount drive, import custom modules
dataset.py, criterion.py, transform.py file from [provided link](https://github.com/bodokaiser/piwise)

In [ ]:
# mount your Google Drive
drive.mount("/content/gdrive", force_remount=True)
# copy source files to this directory, to be able to import it directly
# if these files are copied and imported without any error,
# mounting is successful
!cp "gdrive/My Drive/Project/dataset.py" .
!cp "gdrive/My Drive/Project/criterion.py" .
!cp "gdrive/My Drive/Project/transform.py" .
!cp "gdrive/My Drive/Project/unet.py" .
#!cp "gdrive/My Drive/Project/attention_unet.py"
from dataset import VOC12
from criterion import CrossEntropyLoss2d
from transform import Relabel, ToLabel, Colorize
#from unet import myUnet
import unet
#from attention_unet import AttentionUnet

Mounted at /content/gdrive


## training
the basic structure of this function is also from [provided link](https://github.com/bodokaiser/piwise)

In [ ]:
color_transform = Colorize()
image_transform = ToPILImage()
input_transform = Compose([
    CenterCrop(256), # 256x256 cropped image
    ToTensor(),
    Normalize([.485, .456, .406], [.229, .224, .225]), # mean, std for each channels
])
target_transform = Compose([
    CenterCrop(256), # 256x256 cropped image
    ToLabel(),
    Relabel(255, 21),
])

def train(args, model):
    # set model to training mode
    model.train()
    # prepare criterion
    weight = torch.ones(22)
    weight[0] = 0
    # load data - even though Python notebook doesn't really support CLI arguments,
    # attributes in args will be assigned manually when calling main() function
    loader = DataLoader(VOC12(args.datadir, input_transform, target_transform),
        num_workers=args.num_workers, batch_size=args.batch_size, shuffle=True)
    # use Adam optimizer
    optimizer = Adam(model.parameters())
    if args.cuda:
        criterion = CrossEntropyLoss2d(weight.cuda())
    else:
        criterion = CrossEntropyLoss2d(weight)
    # start training - epoch values start from 1 to make numbers look 'pretty'
    for epoch in range(1, args.num_epochs + 1):
        epoch_loss = []
        for step, (images, labels) in enumerate(loader):
            if args.cuda:
                images = images.cuda()
                labels = labels.cuda()
            inputs = Variable(images)
            targets = Variable(labels)
            outputs = model(inputs)
            # refresh gradient before backprop
            optimizer.zero_grad()
            loss = criterion(outputs, targets[:, 0])
            loss.backward()
            optimizer.step()

            #epoch_loss.append(loss.data[0])
            epoch_loss.append(loss.item())
            if (args.steps_loss > 0) and (step % args.steps_loss == 0):
                average = sum(epoch_loss) / len(epoch_loss)
                print(f'loss: {average} (epoch {epoch}, step {step})')
            if (args.steps_save > 0) and (step % args.steps_save == 0):
                if args.attention:
                    filename = f'AttentionUNet-{epoch:03}-{step:04}.pth'
                else:
                    filename = f'UNet-{epoch:03}-{step:04}.pth'
                torch.save(model.state_dict(), filename)
                print(f'save: {filename} (epoch: {epoch}, step: {step})')

## evaluation

In [ ]:
def evaluate(args, model):
    # set model to evaluation mode
    model.eval()

    image = input_transform(Image.open(args.image))
    label = model(Variable(image, volatile=True).unsqueeze(0))
    label = color_transform(label[0].data.max(0)[1])

    image_transform(label).save(args.label)

## main function


In [ ]:
def main(args):
    # load correct model
    if args.attention == True:
        raise NotImplementedError
        #model = AttentionUnet()
    elif args.attention == False:
        # we don't use residual for non-denoising purposes
        model = unet.Unet(in_channels=NUM_CHANNELS,out_channels=NUM_CLASSES
                          ,use_residual=False, use_norm=True)
    # check if we use GPU
    if args.cuda:
        model = model.cuda()
    # check for mode - training or evaluation?
    if args.mode == 'eval':
        evaluate(args, model)
    elif args.mode == 'train':
        train(args, model)

## actual running
the basic structure of this snippet is also from [provided link](https://github.com/bodokaiser/piwise)

In [ ]:
if __name__ == '__main__':
    # comment out lines from original code that is not used here
    parser = ArgumentParser()
    parser.add_argument('--cuda', action='store_true')
    parser.add_argument('--attention', action='store_true')
    #parser.add_argument('--state')

    subparsers = parser.add_subparsers(dest='mode')
    subparsers.required = True

    parser_eval = subparsers.add_parser('eval')
    parser_eval.add_argument('image')
    parser_eval.add_argument('label')

    parser_train = subparsers.add_parser('train')
    #parser_train.add_argument('--port', type=int, default=80)
    parser_train.add_argument('--datadir', required=True)
    parser_train.add_argument('--num-epochs', type=int, default=32)
    parser_train.add_argument('--num-workers', type=int, default=4)
    parser_train.add_argument('--batch-size', type=int, default=1)
    parser_train.add_argument('--steps-loss', type=int, default=50)
    #parser_train.add_argument('--steps-plot', type=int, default=0)
    parser_train.add_argument('--steps-save', type=int, default=500)
    # since we can't pass CLI arugments in Python notebook,
    # assign variables manually
    # add two spaces between each chunk so that the data directory doesn't get split
    command_line = "--cuda  train  --datadir  gdrive/My Drive/Project/data  --num-epochs  1  --num-workers  4  --batch-size  4  --steps-save  500"
    args = parser.parse_args(command_line.split("  "))
    print(args)
    main(args)

Namespace(attention=False, batch_size=4, cuda=True, datadir='gdrive/My Drive/Project/data', mode='train', num_epochs=1, num_workers=4, steps_loss=50, steps_save=500)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:211: UserWarning: NLLLoss2d has been deprecated. Please use NLLLoss instead as a drop-in replacement and see https://pytorch.org/docs/master/nn.html#torch.nn.NLLLoss for more details.
  warnings.warn("NLLLoss2d has been deprecated. "
/content/criterion.py:13: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.loss(F.log_softmax(outputs), targets)


loss: 3.0050792694091797 (epoch 1, step 0)
save: UNet-001-0000.pth (epoch: 1, step: 0)
loss: 2.958860387989119 (epoch 1, step 50)
loss: 2.903348134295775 (epoch 1, step 100)
loss: 2.891738574236434 (epoch 1, step 150)
loss: 2.85823853098931 (epoch 1, step 200)
loss: 2.8436008544557123 (epoch 1, step 250)
loss: 2.840546433711765 (epoch 1, step 300)
loss: 2.827510713512062 (epoch 1, step 350)
loss: 2.8133812738475656 (epoch 1, step 400)
loss: 2.798139847831557 (epoch 1, step 450)
loss: 2.790436963359277 (epoch 1, step 500)
save: UNet-001-0500.pth (epoch: 1, step: 500)
loss: 2.7839086546871927 (epoch 1, step 550)
loss: 2.7703526426670755 (epoch 1, step 600)
loss: 2.7653924054630705 (epoch 1, step 650)
loss: 2.7572788656523155 (epoch 1, step 700)


In [ ]:
!ls /content/

criterion.py  gdrive	   sample_data	 unet.py
dataset.py    __pycache__  transform.py
